In [88]:
import pandas as pd 
import tensorflow
from tensorflow import keras
from keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

df = pd.read_csv("Churn_Modelling.csv")

In [89]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [90]:
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [91]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [92]:
le = LabelEncoder()
for col in df.select_dtypes("object"):
    df[col] = le.fit_transform(df[col])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


In [93]:
Y = df["Exited"]
X = df.drop(["Exited"], axis=1)
Xtrain, Xtest,ytrain,ytest = train_test_split(X,Y, test_size=0.2, shuffle=True, random_state=42)

In [94]:
#scaling es muy importante para gardient descent en NN, SVM, distance measurementes, knn 
scaler = MinMaxScaler()
#important to apply it after splitting bc it affects data leakage and overfitting if combined with xtest , por eso debe ser independiente
Xtrain = scaler.fit_transform(Xtrain)
#se aplican los parameters learned from the training set
Xtest = scaler.transform(Xtest)


In [95]:
import tensorflow as tf
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.utils import class_weight
import numpy as np


unique_classes = np.unique(ytrain)
print(f"Unique classes: {unique_classes}")

# Calculate class weights
classWeights = class_weight.compute_class_weight('balanced', classes=unique_classes, y=ytrain)
classWeights = dict(enumerate(classWeights))

Unique classes: [0 1]


In [96]:
classWeights

{0: 0.6293266205160478, 1: 2.4330900243309004}

In [97]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='recall',    # Metric to monitor
    patience=3,            # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

model = Sequential()
model.add(Dense(units=500,  activation="relu"))
model.add(Dense(units=400, activation="relu"))
model.add(Dense(units=400, activation="relu"))
model.add(Dense(units=100, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["recall"])
model.fit(x=Xtrain, y=ytrain, epochs=20, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20


200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4774 - recall: 0.0179 - val_loss: 0.4503 - val_recall: 0.4689
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4027 - recall: 0.3353 - val_loss: 0.3784 - val_recall: 0.3727
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3560 - recall: 0.3932 - val_loss: 0.3631 - val_recall: 0.3820
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3481 - recall: 0.4506 - val_loss: 0.3655 - val_recall: 0.3665
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3389 - recall: 0.4308 - val_loss: 0.3525 - val_recall: 0.3789
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3525 - recall: 0.4424 - val_loss: 0.3624 - val_recall: 0.3758
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3507 - recall: 0.4419 - val_loss: 0.3629 - val_recall: 0.3634
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3378 - recall: 0.4432 - val_loss: 0.3565 - val_recall: 0.4099
Epoch 9/20
200/200 

In [98]:
loss, recall = model.evaluate(Xtest, ytest)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3430 - recall: 0.4334


In [99]:
ypred = (model.predict(Xtest) > 0.5).astype("int32")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [100]:
ypred

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [101]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92      1607
           1       0.77      0.45      0.57       393

    accuracy                           0.86      2000
   macro avg       0.82      0.71      0.74      2000
weighted avg       0.86      0.86      0.85      2000

